In [9]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat or 'tfidf' in feat:
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/fasttext_cnn2d_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v2_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cudnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_gru_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_lstm_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn2d_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cudnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_gru_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_lstm_v1_4_feat.pkl
(159571, 6) (15

In [10]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss'):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        d_test = test_x
        
        # share params
        params = {
                'application': 'binary',
                #'num_leaves': 8,
                #'lambda_l1': 1,
                'lambda_l2': 1.0,
                'max_depth': 3,
                #'scale_pos_weight':0.9,
                'metric': met, # or auc
                'data_random_seed': 2,
                'learning_rate':lr,
#                 'bagging_fraction': bagging_fraction,
#                 'bagging_freq':bag_frec,
                'feature_fraction': feature_fraction,
            
                }
        if met == 'auc':
            s_round = 100
        else:
            s_round = 50
        # train for each class
        for i in range(6):
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(i)
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        train_auc_l = train_auc_l/6
        val_auc_l = val_auc_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        print('this fold auc train',train_auc_l,'auc val',val_auc_l)
        
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        all_train_auc_l += train_auc_l/k
        all_val_auc_l += val_auc_l/k
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred

print('done')

done


In [11]:

lgb_res = simple_ens('lgb',10,233,0.05,0.6)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# all loss avg 0.0335311567862 0.036750116241
# all auc avg 0.993683089291 0.990779711364 PUB 9862

# change params lr to 0.05, rnd 42
# all loss avg 0.0331165975108 0.036782889115 all auc avg 0.993888891968 0.99078987792

# rm bagging, rnd 233
# all loss avg 0.0330784234373 0.036800843461 all auc avg 0.993977317117 0.990582906649

# fix lr, mnb feat, no bagging, rnd 233
# all loss avg 0.032247976023 0.0361280397445 all auc avg 0.994391483047 0.991118473891

# add 2 feats
# all loss avg 0.0322831057545 0.0359632999094 all auc avg 0.994383324993 0.991201300627 PUB 9866

# add other feats, 20 dims
# all loss avg 0.0322929412255 0.0359087268902 all auc avg 0.994411779666 0.991163662181

# add 7 base fasttext NN models
# all loss avg 0.0319116484218 0.0358161833583 all auc avg 0.994546891137 0.991249510282 PUB 9866

# rm tfidf test
# all loss avg 0.0319555637279 0.0357756335619 all auc avg 0.994512718368 0.991251471241

0
ls 0.0674956971593 0.0749572922159 auc 0.991437975178 0.98854546507
1
ls 0.0182665004623 0.0206443054284 auc 0.993933653303 0.992121220444
2
ls 0.0353057648081 0.0400445624363 auc 0.996191283288 0.994634575842
3
ls 0.00366903482398 0.00678662314936 auc 0.999454645613 0.997943406317
4
ls 0.0514496577762 0.0564793200085 auc 0.991130416038 0.98809529958
5
ls 0.0153521241644 0.0196537241074 auc 0.995047253942 0.988926390821
this fold avg train 0.0319231298657 avg val 0.036427637891
this fold auc train 0.994532537894 auc val 0.991711059679
0
ls 0.0694049451651 0.0773546340312 auc 0.990834481195 0.986829311029
1
ls 0.018409019844 0.0189733387294 auc 0.993801905864 0.993750308123
2
ls 0.0351392087998 0.0347716990827 auc 0.996286977757 0.996104300774
3
ls 0.00489791312486 0.00755040913573 auc 0.998704975143 0.994375667693
4
ls 0.0495186971221 0.0486650442867 auc 0.991975064413 0.991661089768
5
ls 0.0154797784855 0.0164968992392 auc 0.995077733119 0.988684725642
this fold avg train 0.03214159

In [12]:
# adj lr, feat_frac, bag_frac
for lr in [0.05]:
    for c1 in [0.6,0.7,0.8]:
        lgb_res = simple_ens('lgb',5,233,lr,c1)
        sample_submission = pd.read_csv("../input/sample_submission.csv")
        sample_submission[list_classes] = lgb_res
        fname = "../results/lgb_csv_fold5_{}_{}.gz".format(lr,c1)
        sample_submission.to_csv(fname, index=False, compression='gzip')
        print(fname)
        print(sample_submission.head())
        print('save done')
        break

# lr, feat_frac, bag_frac, bag_freq=40
# 0.03   0.7   0.7       all loss avg 0.0322530666235 0.0369762464226 all auc avg 0.994271044586 0.990285134782
# 0.03   0.7   0.8       all loss avg 0.0322971080147 0.0369559723174 all auc avg 0.994334369995 0.990306416634
# 0.03   0.8   0.7       all loss avg 0.0322229511884 0.0369948009383 all auc avg 0.994234840050 0.990368331216
# 0.03   0.8   0.8       all loss avg 0.0324206645596 0.0369651316109 all auc avg 0.994257933028 0.990295813203
# 0.05   0.7   0.7       all loss avg 0.0318163169078 0.0370403091733 all auc avg 0.994337086137 0.990254302965

# bag_freq = 3
# 0.05   0.6   0.6       all loss avg 0.0329345052647 0.0368498383873 all auc avg 0.9939819323 0.990664190969
# 0.05   0.6   0.7       all loss avg 0.0326017869132 0.0368044492709 all auc avg 0.994163898916 0.990696436189
# 0.05   0.6   0.8       all loss avg 0.0324589992208 0.0368163370811 all auc avg 0.994233438434 0.990618323557
# 0.05   0.7   0.6       all loss avg 0.0331541550186 0.0368670775083 all auc avg 0.993940708877 0.990597573572
# 0.05   0.7   0.7       all loss avg 0.0325753698442 0.0368285766463 all auc avg 0.994218398119 0.990663705936
# 0.05   0.7   0.8       all loss avg 0.0325779317956 0.0368281454058 all auc avg 0.994220915054 0.990630132284
# 0.05   0.8   0.6       all loss avg 0.0333336885412 0.0368817399664 all auc avg 0.993874933751 0.990630976344
# 0.05   0.8   0.7       all loss avg 0.0329593178033 0.0368484307605 all auc avg 0.994000316129 0.99065277921
# 0.05   0.8   0.8       all loss avg 0.0328682234385 0.0368555923112 all auc avg 0.994090826609 0.990708113059

# rm bagging
# 0.05   0.6             all loss avg 0.0328499864930 0.0368782748851 all auc avg 0.994090121326 0.990573017994   
# add 2 feats
# all loss avg 0.0319117001045 0.0360410792425 all auc avg 0.994541896459 0.991241823846
# rm pos_scale_weights, change l2 lambda to 1
# all loss avg 0.0316907179553 0.0359932860253 all auc avg 0.994616385099 0.991125111598

# add other feats
# all loss avg 0.031778552123 0.0359415940753 all auc avg 0.99463706224 0.991188429702

KeyboardInterrupt: 